<a href="https://colab.research.google.com/github/AaryanJethva7/NMIMS-NLP/blob/Home-Work/NLP_TopicModelling_EnronEmails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [7]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
df=pd.read_csv('/gdrive/My Drive/emails.csv')

In [9]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [11]:
import nltk
import re
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english')) 
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
nltk.download('wordnet')
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def pipeline(text):
  text=re.sub('[^a-zA-Z]'," ",str(text))
  text=text.lower()
  tokens=word_tokenize(text)
  nostop=[words for words in tokens if not words in stop_words]
  nostop=[lemmatizer.lemmatize(word) for word in tokens]
  str1 = " "  
  return (str1.join(nostop))

In [0]:
df=df[0:500]

In [14]:
df.shape

(500, 2)

In [0]:
df['Clean Message']=df['message'].apply(pipeline)

In [16]:
df.head()

,file,message,Clean Message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,message id javamail evans thyme date mon may p...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,message id javamail evans thyme date fri may p...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,message id javamail evans thyme date wed oct p...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,message id javamail evans thyme date mon oct p...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,message id javamail evans thyme date thu aug p...


In [0]:
from sklearn.decomposition import NMF

In [0]:
model=tfidf.fit_transform(df['Clean Message'])

In [0]:
model=pd.DataFrame(model.toarray(),columns=tfidf.get_feature_names())

In [0]:
model_nmf=NMF(n_components=5, init='random', random_state=0)

In [0]:
W = model_nmf.fit_transform(model)

In [23]:
W

array([[0.        , 0.08874886, 0.00167917, 0.00264165, 0.00494311],
       [0.04685546, 0.02167115, 0.00228651, 0.00533303, 0.0030589 ],
       [0.        , 0.06360675, 0.        , 0.00377793, 0.        ],
       ...,
       [0.03132704, 0.04245319, 0.00065958, 0.        , 0.00087554],
       [0.        , 0.0665954 , 0.00897699, 0.        , 0.01586926],
       [0.04892796, 0.03382322, 0.        , 0.        , 0.        ]])

In [0]:
H = model_nmf.components_

In [25]:
H

array([[1.09157009e-03, 1.62710875e-02, 0.00000000e+00, ...,
        5.28081549e-03, 4.39072975e-03, 1.12019806e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.18574536e-04, 0.00000000e+00],
       [3.26238373e-04, 1.81587824e-03, 0.00000000e+00, ...,
        3.48371025e-03, 6.09248311e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.33011493e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.17304040e-03, 1.29395485e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [0]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [0]:
lda = LatentDirichletAllocation(n_components=5, random_state=0)

In [29]:
lda.fit(model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [30]:
lda.components_

array([[0.20448145, 0.30831981, 0.20629485, ..., 0.26263556, 0.28234955,
        0.24000765],
       [0.20000894, 0.2000446 , 0.2000237 , ..., 0.20003435, 0.21548475,
        0.20004573],
       [0.20000852, 0.20684416, 0.22796243, ..., 0.20003186, 0.20004825,
        0.2459538 ],
       [0.20745229, 0.20004281, 0.20002225, ..., 0.20003264, 0.20005012,
        0.2000443 ],
       [0.20000935, 0.22705362, 0.20002474, ..., 0.20003538, 0.20005063,
        0.20004667]])